<a href="https://colab.research.google.com/github/Kwangbin/data_01/blob/master/coffee_store_eda_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 소상공인시장진흥공단 상가업소정보로 스타벅스, 이디야 위치 분석하기
* https://www.data.go.kr/dataset/15012005/fileData.do
* [[비즈&빅데이터]스타벅스 '쏠림' vs 이디야 '분산'](http://news.bizwatch.co.kr/article/consumer/2018/01/19/0015)

In [0]:
# warning 메시지 제거, warning 메시지는 라이브러리 업데이트나 사용법에 대한 안내 등이 있습니다.
# 코딩을 처음 시작할 때는  warning 메시지가 나오면 당황하실 수도 있어서 제거를 하고 보도록 합니다.
# warning 메시지는 제거하고 보셔도 되지만 Error 메시지를 꼭 고쳐주셔야 합니다.
import warnings
warnings.filterwarnings('ignore')

## 필요한 라이브러리 불러오기

In [0]:
#pd, np, sns등 공식 명칭을 사용 하는게 오류를 줄임
import pandas as pd
import numpy as np
import seaborn as sns
# 지도 시각화를 위해
import folium

# 노트북 안에서 그래프를 표시하기 위해
%matplotlib inline

In [0]:
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
# Window 의 한글 폰트 설정
# plt.rc('font',family='Malgun Gothic')
# Mac 의 한글 폰트 설정
plt.rc('font', family='Malgun Gothic') 
plt.rc('axes', unicode_minus=False)

set_matplotlib_formats('retina')

# 그래프가 노트북 안에 보이게 하기 위해
%matplotlib inline

## Colab 에서 실행을 위한 코드

* 아래의 코드는 google colaboratory 에서 실행을 위한 코드로 로컬 아나콘다에서는 주석처리합니다.
* google colaboratory 에서는 주석을 풀고 폰트 설정과 csv 파일을 불러옵니다.

In [9]:
# 나눔고딕 설치
!apt -qq -y install fonts-nanum > /dev/null

import matplotlib.font_manager as fm

fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
fm._rebuild()

# 그래프에 retina display 적용
%config InlineBackend.figure_format = 'retina'

# Colab 의 한글 폰트 설정
plt.rc('font', family='NanumBarunGothic') 

In [0]:
# 구글 드라이브에서 csv 파일을 읽어오기 위해 gauth 인증
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# PyDrive client 인증
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [11]:
# 공유 가능한 링크로 파일 가져오기
url = 'https://drive.google.com/open?id=1xvJs9ZxUHoBBAUXq1IWEaRXgcop8KjrT'
id = url.split('=')[1]
print(id)
downloaded = drive.CreateFile({'id':id}) 
# data 폴더에 파일을 관리하며, 폴더가 없다면 만들어서 파일을 관리하도록 한다.
%mkdir data
downloaded.GetContentFile('data/store_201906_01.csv')  

1xvJs9ZxUHoBBAUXq1IWEaRXgcop8KjrT
mkdir: cannot create directory ‘data’: File exists


## 데이터 불러오기
* 데이터 다운로드 위치 : https://www.data.go.kr/dataset/15012005/fileData.do
* 공공데이터 포털에서 소상공인시장진흥공단 상가업소정보를 다운로드 받아 사용했습니다.
* 구글드라이브에서 받기 : https://drive.google.com/open?id=1xvJs9ZxUHoBBAUXq1IWEaRXgcop8KjrT

In [12]:
# 데이터를 불러와 df_store 라는 변수에 담아주세요.
df_store = pd.read_csv("data/store_201906_01.csv")
df_store.head()

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,표준산업분류명,시도코드,시도명,시군구코드,시군구명,행정동코드,행정동명,법정동코드,법정동명,지번코드,대지구분코드,대지구분명,지번본번지,지번부번지,지번주소,도로명코드,도로명,건물본번지,건물부번지,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
0,19911397,커피빈코리아선릉로93길점,코리아선릉로93길점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,비알콜 음료점업,11,서울특별시,11680,강남구,1168064000,역삼1동,1168010100,역삼동,1168010100206960042,1,대지,696,42.0,서울특별시 강남구 역삼동 696-42,116804166502,서울특별시 강남구 선릉로93길,6,NaN,1168010100106960042022041,NaN,서울특별시 강남구 선릉로93길 6,135080,6149.0,NaN,1,NaN,127.047883,37.505675
1,19911801,프로포즈,NaN,Q,음식,Q09,유흥주점,Q09A01,호프/맥주,I56219,기타 주점업,11,서울특별시,11545,금천구,1154563000,독산3동,1154510200,독산동,1154510200201620001,1,대지,162,1.0,서울특별시 금천구 독산동 162-1,115453117003,서울특별시 금천구 가산로,34,6.0,1154510200101620001017748,NaN,서울특별시 금천구 가산로 34-6,153010,8545.0,NaN,1,NaN,126.899220,37.471711
2,19912201,싱싱커피&토스트,NaN,Q,음식,Q07,패스트푸드,Q07A10,토스트전문,I56192,"피자, 햄버거, 샌드위치 및 유사 음식점업",26,부산광역시,26530,사상구,2653062000,괘법동,2653010400,괘법동,2653010400205780000,1,대지,578,NaN,부산광역시 사상구 괘법동 578,265303139004,부산광역시 사상구 괘감로,37,NaN,2653010400105780000002037,산업용품유통상가,부산광역시 사상구 괘감로 37,617726,46977.0,NaN,1,26,128.980455,35.159774
3,19905471,와라와라호프,NaN,Q,음식,Q09,유흥주점,Q09A01,호프/맥주,I56219,기타 주점업,11,서울특별시,11680,강남구,1168060000,대치1동,1168010600,대치동,1168010600206040000,1,대지,604,NaN,서울특별시 강남구 대치동 604,116802000003,서울특별시 강남구 남부순환로,2933,NaN,1168010600106040000014378,대치상가,서울특별시 강남구 남부순환로 2933,135280,6280.0,NaN,1,NaN,127.061026,37.493922
4,19932756,가락사우나내스낵,NaN,F,생활서비스,F09,대중목욕탕/휴게,F09A02,사우나/증기탕/온천,S96121,욕탕업,11,서울특별시,11710,송파구,1171060000,석촌동,1171010500,석촌동,1171010500202560000,1,대지,256,NaN,서울특별시 송파구 석촌동 256,117103123001,서울특별시 송파구 가락로,71,NaN,1171010500102560005010490,NaN,서울특별시 송파구 가락로 71,138846,5690.0,NaN,1,NaN,127.104071,37.500249


## 데이터 크기 보기

In [13]:
df_store.shape

(565016, 39)

## info 보기

In [14]:
df_store.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 565016 entries, 0 to 565015
Data columns (total 39 columns):
상가업소번호       565016 non-null int64
상호명          565015 non-null object
지점명          75524 non-null object
상권업종대분류코드    565016 non-null object
상권업종대분류명     565016 non-null object
상권업종중분류코드    565016 non-null object
상권업종중분류명     565016 non-null object
상권업종소분류코드    565016 non-null object
상권업종소분류명     565016 non-null object
표준산업분류코드     532064 non-null object
표준산업분류명      532064 non-null object
시도코드         565016 non-null int64
시도명          565016 non-null object
시군구코드        565016 non-null int64
시군구명         565016 non-null object
행정동코드        565016 non-null int64
행정동명         565016 non-null object
법정동코드        565016 non-null int64
법정동명         565016 non-null object
지번코드         565016 non-null int64
대지구분코드       565016 non-null int64
대지구분명        565016 non-null object
지번본번지        565016 non-null int64
지번부번지        467893 non-null float64
지번주소         565016 non-null obje

## 결측치 보기

In [15]:
#상호명과 지점명이 틀리는 결측치 발생
df_store.isnull().sum()

상가업소번호            0
상호명               1
지점명          489492
상권업종대분류코드         0
상권업종대분류명          0
상권업종중분류코드         0
상권업종중분류명          0
상권업종소분류코드         0
상권업종소분류명          0
표준산업분류코드      32952
표준산업분류명       32952
시도코드              0
시도명               0
시군구코드             0
시군구명              0
행정동코드             0
행정동명              0
법정동코드             0
법정동명              0
지번코드              0
대지구분코드            0
대지구분명             0
지번본번지             0
지번부번지         97123
지번주소              0
도로명코드             0
도로명               0
건물본번지             0
건물부번지        494164
건물관리번호            0
건물명          303808
도로명주소             0
구우편번호             0
신우편번호             9
동정보          514802
층정보          220463
호정보          481880
경도                0
위도                0
dtype: int64

## 사용하지 않는 컬럼 제거하기

In [0]:
# 결측치 문제로 결측치 제거
drop_columns = ["상권업종중분류코드", "상권업종소분류코드", 
                "건물관리번호", "표준산업분류코드", "표준산업분류명", 
                "지번부번지", "층정보", "건물명", "호정보", 
                "지점명", "건물부번지", "동정보"]

In [18]:
df_store[drop_columns]

,상권업종중분류코드,상권업종소분류코드,건물관리번호,표준산업분류코드,표준산업분류명,지번부번지,층정보,건물명,호정보,지점명,건물부번지,동정보
0,Q12,Q12A01,1168010100106960042022041,I56220,비알콜 음료점업,42.0,1,NaN,NaN,코리아선릉로93길점,NaN,NaN
1,Q09,Q09A01,1154510200101620001017748,I56219,기타 주점업,1.0,1,NaN,NaN,NaN,6.0,NaN
2,Q07,Q07A10,2653010400105780000002037,I56192,"피자, 햄버거, 샌드위치 및 유사 음식점업",NaN,1,산업용품유통상가,26,NaN,NaN,NaN
3,Q09,Q09A01,1168010600106040000014378,I56219,기타 주점업,NaN,1,대치상가,NaN,NaN,NaN,NaN
4,F09,F09A02,1171010500102560005010490,S96121,욕탕업,NaN,1,NaN,NaN,NaN,NaN,NaN
5,Q07,Q07A10,1121510300100770047020647,I56192,"피자, 햄버거, 샌드위치 및 유사 음식점업",47.0,1,NaN,NaN,NaN,NaN,NaN
6,Q04,Q04A01,1150010800100450031005883,I56194,분식 및 김밥 전문점,89.0,1,NaN,NaN,NaN,NaN,NaN
7,Q07,Q07A01,1114016200103720095000001,I56192,"피자, 햄버거, 샌드위치 및 유사 음식점업",95.0,1,현진빌딩,NaN,약수점,NaN,NaN
8,Q01,Q01A02,1150010300110860003013118,I56111,한식 음식점업,3.0,1,NaN,NaN,NaN,NaN,NaN
9,Q03,Q03A03,1171010200100070028000237,I56113,일식 음식점업,28.0,1,현대타워아파트,NaN,NaN,19.0,NaN


In [0]:
# 아래 발생 하는 자료의 예제를 실행 해보면 이행 하기 쉽다.
df_store.drop?


In [20]:
#컬럼 사라지지 않음 변수에 다시 넣어야함
df_store.drop(drop_columns, axis=1)

,상가업소번호,상호명,상권업종대분류코드,상권업종대분류명,상권업종중분류명,상권업종소분류명,시도코드,시도명,시군구코드,시군구명,행정동코드,행정동명,법정동코드,법정동명,지번코드,대지구분코드,대지구분명,지번본번지,지번주소,도로명코드,도로명,건물본번지,도로명주소,구우편번호,신우편번호,경도,위도
0,19911397,커피빈코리아선릉로93길점,Q,음식,커피점/카페,커피전문점/카페/다방,11,서울특별시,11680,강남구,1168064000,역삼1동,1168010100,역삼동,1168010100206960042,1,대지,696,서울특별시 강남구 역삼동 696-42,116804166502,서울특별시 강남구 선릉로93길,6,서울특별시 강남구 선릉로93길 6,135080,6149.0,127.047883,37.505675
1,19911801,프로포즈,Q,음식,유흥주점,호프/맥주,11,서울특별시,11545,금천구,1154563000,독산3동,1154510200,독산동,1154510200201620001,1,대지,162,서울특별시 금천구 독산동 162-1,115453117003,서울특별시 금천구 가산로,34,서울특별시 금천구 가산로 34-6,153010,8545.0,126.899220,37.471711
2,19912201,싱싱커피&토스트,Q,음식,패스트푸드,토스트전문,26,부산광역시,26530,사상구,2653062000,괘법동,2653010400,괘법동,2653010400205780000,1,대지,578,부산광역시 사상구 괘법동 578,265303139004,부산광역시 사상구 괘감로,37,부산광역시 사상구 괘감로 37,617726,46977.0,128.980455,35.159774
3,19905471,와라와라호프,Q,음식,유흥주점,호프/맥주,11,서울특별시,11680,강남구,1168060000,대치1동,1168010600,대치동,1168010600206040000,1,대지,604,서울특별시 강남구 대치동 604,116802000003,서울특별시 강남구 남부순환로,2933,서울특별시 강남구 남부순환로 2933,135280,6280.0,127.061026,37.493922
4,19932756,가락사우나내스낵,F,생활서비스,대중목욕탕/휴게,사우나/증기탕/온천,11,서울특별시,11710,송파구,1171060000,석촌동,1171010500,석촌동,1171010500202560000,1,대지,256,서울특별시 송파구 석촌동 256,117103123001,서울특별시 송파구 가락로,71,서울특별시 송파구 가락로 71,138846,5690.0,127.104071,37.500249
5,19964615,석봉토스트,Q,음식,패스트푸드,토스트전문,11,서울특별시,11215,광진구,1121586000,구의2동,1121510300,구의동,1121510300200770047,1,대지,77,서울특별시 광진구 구의동 77-47,112154112444,서울특별시 광진구 자양로39길,20,서울특별시 광진구 자양로39길 20,143200,4992.0,127.088387,37.549245
6,19975850,허술한집,Q,음식,분식,라면김밥분식,11,서울특별시,11500,강서구,1150062000,공항동,1150010800,공항동,1150010800200450089,1,대지,45,서울특별시 강서구 공항동 45-89,115004145181,서울특별시 강서구 공항대로3길,9,서울특별시 강서구 공항대로3길 9,157812,7619.0,126.809957,37.562013
7,19977290,피자마루,Q,음식,패스트푸드,피자전문,11,서울특별시,11140,중구,1114063500,약수동,1114016200,신당동,1114016200203720095,1,대지,372,서울특별시 중구 신당동 372-95,111404103114,서울특별시 중구 동호로7길,37,서울특별시 중구 동호로7길 37,100450,4596.0,127.010861,37.552153
8,19979960,홍능갈비,Q,음식,한식,갈비/삼겹살,11,서울특별시,11500,강서구,1150054000,화곡1동,1150010300,화곡동,1150010300210860003,1,대지,1086,서울특별시 강서구 화곡동 1086-3,115004145499,서울특별시 강서구 화곡로18길,31,서울특별시 강서구 화곡로18길 31,157010,7718.0,126.836078,37.538927
9,19983535,초밥왕,Q,음식,일식/수산물,음식점-초밥전문,11,서울특별시,11710,송파구,1171071000,잠실6동,1171010200,신천동,1171010200200070028,1,대지,7,서울특별시 송파구 신천동 7-28,117103123023,서울특별시 송파구 올림픽로,293,서울특별시 송파구 올림픽로 293-19,138735,5510.0,127.102490,37.515149


In [0]:
df_store= df_store.drop(drop_columns, axis=1)

In [22]:
df_store.shape

(565016, 27)

## 일부 컬럼만 보기

In [23]:
# 상권업종대분류명 컬럼만 보기
df_store["상권업종대분류명"].head()

0       음식
1       음식
2       음식
3       음식
4    생활서비스
Name: 상권업종대분류명, dtype: object

In [24]:
# 상권업종소분류명 컬럼만 보기
df_store["상권업종소분류명"].head()

0    커피전문점/카페/다방
1          호프/맥주
2          토스트전문
3          호프/맥주
4     사우나/증기탕/온천
Name: 상권업종소분류명, dtype: object

In [25]:
# 상권업종대분류명, 상권업종소분류명 컬럼만 보기
df_store["상권업종대분류명", "상권업종소분류명"]

KeyError: ignored

In [0]:
# 제거하는 방법도 있지만 사용할 컬럼만 남겨두는 방법도 있습니다.

# 위에 있는 컬럼만 따로 불러옵니다.

In [26]:
#두개 이상의 항목 가져올때는 [] 리스트 형태로 가져와야 한다.
df_store[["상권업종대분류명", "상권업종소분류명"]].head()

,상권업종대분류명,상권업종소분류명
0,음식,커피전문점/카페/다방
1,음식,호프/맥주
2,음식,토스트전문
3,음식,호프/맥주
4,생활서비스,사우나/증기탕/온천


In [27]:
df_store[["상권업종대분류명", "상권업종소분류명"]].sample()

,상권업종대분류명,상권업종소분류명
184993,음식,파전전문


## 일부 텍스트가 들어가는 데이터만 가져오기

In [0]:
# 결측치 문제로 결측치 제거
drop_columns = df_columns = ['상가업소번호', '상호명', '상권업종대분류명', 
       '상권업종중분류명',  '상권업종소분류명', 
       '시도명', '시군구명', '행정동명', '법정동명', 
       '지번주소', '도로명코드', '도로명주소', 
       '구우편번호', '신우편번호', '경도', '위도']

In [0]:
df_store=df_store[df_columns].copy()

In [33]:
df_store.shape

(565016, 16)

In [0]:
# 커피라는 텍스트가 들어가는 상권업소중소분류명만 가져옵니다.
# Pandas Series의 str.contains 기능을 사용해 가져올 수 있습니다.
# 가져온 데이터를 coffee 라는 데이터프레임에 담아봅니다.

In [0]:
# 상호명으로 봤을 때 커피점의 수 보기

In [35]:
coffee=df_store[df_store['상권업종소분류명'].str.contains("커피")]
coffee.shape

(23103, 16)

## 특정 상호만 가져오기
* 여기에서는 스타벅스 상호를 가져와서 실습합니다.
* 위에서 커피를 가져왔던 내용을 활용해 봅니다.

In [36]:
# 상호명을 보면 겹치는 문자가 많습니다. 스타벅스의 unique한 상호명만 가져옵니다.
coffee["상호명"].unique()

array(['커피빈코리아선릉로93길점', '스완카페트', '왕실', ..., '놀러오시개', '뉴욕보틀', '블루빈로스터스커피'],
      dtype=object)

In [37]:
coffee["상호명"].nunique()

15205

In [41]:
coffee.loc[coffee["상호명"].str.contains("스타벅스"),"상호명"].head()

1219               스타벅스
1946          스타벅스종로3가점
2840               스타벅스
14939    스타벅스커피여의도IFC1F
14951              스타벅스
Name: 상호명, dtype: object

## 시도별 데이터 보기
* 시도와 구군 데이터가 있습니다. 어느 시도의 데이터가 있는지 봅니다.

In [43]:
df_store["시도명"]

0         서울특별시
1         서울특별시
2         부산광역시
3         서울특별시
4         서울특별시
5         서울특별시
6         서울특별시
7         서울특별시
8         서울특별시
9         서울특별시
10        서울특별시
11        서울특별시
12        서울특별시
13        서울특별시
14        서울특별시
15        서울특별시
16        서울특별시
17        서울특별시
18        서울특별시
19        서울특별시
20        서울특별시
21        서울특별시
22        서울특별시
23        서울특별시
24        서울특별시
25        서울특별시
26        서울특별시
27        서울특별시
28        부산광역시
29        부산광역시
          ...  
564986    서울특별시
564987    서울특별시
564988    서울특별시
564989    부산광역시
564990    서울특별시
564991    서울특별시
564992    서울특별시
564993    서울특별시
564994    부산광역시
564995    부산광역시
564996    서울특별시
564997    서울특별시
564998    서울특별시
564999    부산광역시
565000    서울특별시
565001    서울특별시
565002    부산광역시
565003    서울특별시
565004    부산광역시
565005    부산광역시
565006    서울특별시
565007    부산광역시
565008    서울특별시
565009    부산광역시
565010    서울특별시
565011    부산광역시
565012    서울특별시
565013    부산광역시
565014    부산광역시
565015    부산광역시
Name: 시도명, Length: 56501

In [45]:
# 시도에 결측치가 있는지 다시 봅니다.
df_store["시도명"].isnull().sum()

0

In [46]:
# 어떤 시도가 있는지 봅니다.
# 다른 지역을 참고하고 싶다면 같은 폴더에 있는 다른 파일을 참고해 주세요.
# Pandas Series의 value_counts 기능을 사용합니다.
df_store["시도명"].value_counts()

서울특별시    401071
부산광역시    163945
Name: 시도명, dtype: int64

In [0]:
# 시군구명의 unique 값만 봅니다.


In [0]:
# coffee 데이터프레임에서 "상권업종중분류명" 을 봅니다. 
# Pandas Series의 value_counts 기능을 사용합니다.


In [0]:
# 경도와 위도가 있습니다. 데이터를 그려봅니다.
# Pandas의 plot.scatter 기능을 활용합니다.
# df_store[:1000]


## 서울만 따로 보기

In [0]:
df_seoul = df_store.loc[df_store['시도명'].str.startswith('서울')].copy()
df_seoul.shape

In [0]:
# df_seoul 에서 상권업종대분류명의 value_counts 를 봅니다.


In [0]:
# describe 를 활용해 요약해 봅니다.
# np.object를 사용하면 object 데이터의 요약도 볼 수 있습니다.



In [0]:
# include=np.number 로 수치 데이터만 요약합니다.
# 여기에서는 위도와 경도만 불러와서 봅니다.


### 분류별 countplot 그리기

In [0]:
# seaborn 의 sns.countplot을 활용해서 상권업종대분류명 의 수량을 세어 막대그래프를 그려봅니다.



In [0]:
# 아래의 코드를 활용해서 음식이라는 텍스트가 들어간 데이터만 df_food 라는 데이터프레임에 저장합니다.
# df_seoul['상권업종대분류명'].str.contains('음식')

# df_food 라는 데이터프레임을 만들어 주세요.

In [0]:
# countplot 을 그리면 해당 카테코리에 해당되는 데이터의 수를 카운트해서 시각화 합니다.
# 상권업종중분류명 으로 countplot을 그려주세요.



In [0]:
# x, y축을 바꿔서 그리게 되면 위에서 글씨가 겹쳐 보이는 것을 보기 좋게 만들 수 있습니다.
# 위 그래프의 x, y축을 바꿔주세요.



## 특정 업종만 가져오기

In [0]:
# 특정 업종만 가져오고 싶다면
# df_food['상권업종중분류명'] == '패스트푸드' 를 활용해서 df_fast_food 라는 데이터 프레임을 만들어 봅니다.



In [0]:
# 해당 데이터에서 특정 컬럼만 가져오고 싶은데 컬럼명이 생각나지 않는다면 컬럼명만 가져와 봅니다.


In [0]:
# df_fast_food['상호명'] 의 unique 값만 가져옵니다.



## Folium 사용예제
http://nbviewer.jupyter.org/github/python-visualization/folium/tree/master/examples/

## 스타벅스, 이디야 위치 분석
* 상호명에서 브랜드명을 추출합니다.
* 대소문자가 섞여 있을 수도 있기 때문에 대소문자를 변환해 줍니다.
* 오타를 방지하기 위해 스타벅스의 영문명은 STARBUCKS, 이디야는 EDIYA 입니다.

In [0]:
# 상호명_소문자라는 새로운 컬럼에 df_seoul['상호명'].str.lower()로 모든 영문자를 소문자로 만들어 저장해 주세요.


In [0]:
# ediya 라는 상호가 들어가는 데이터만 가져와 주세요. 위에서 만든 상호명_소문자 컬럼을 사용합니다.
# df_seoul['상호명_소문자']


In [0]:
# '스타벅스|starbucks'라는 상호가 들어가는 데이터만 가져와 주세요. 위에서 만든 상호명_소문자 컬럼을 사용합니다.
# 참고로 | 연산자는 or를 의미합니다.

# df_seoul['상호명_소문자']


In [0]:
# '이디야|ediya'라는 상호가 들어가는 데이터만 가져와 주세요. 위에서 만든 상호명_소문자 컬럼을 사용합니다.
# 참고로 | 연산자는 or를 의미합니다.



In [0]:
# '스타벅스|starbucks|이디야|ediya 라는 상호가 들어가는 데이터만 가져와 주세요. 
# 위에서 만든 상호명_소문자 컬럼을 사용합니다.
# 결과를 df_cafe에 저장합니다.

# df_seoul['상호명_소문자']

In [0]:
# 브랜드명 이라는 컬럼을 만듭니다.
# 스타벅스만 가져와서 저장합니다.
# df_cafe['상호명_소문자'].str.contains('스타벅스|starbucks')를 활용합니다.



In [0]:
# ~은 not을 의미합니다. 스타벅스가 아닌 데이터는 이디야로 넣어주어도 되지만
# 아래 코드처럼 결측치를 이디야로 채워줘도 괜찮습니다.
# df_cafe.loc[~df_cafe['상호명_소문자'].str.contains('스타벅스|starbucks|STARBUCKS'), '브랜드명'] = '이디야'

In [0]:
# df_cafe['브랜드명'] = df_cafe['브랜드명'].fillna('이디야')

In [0]:
# 혹시라도 상호명에 결측치가 있는지 확인합니다.
# df_cafe['브랜드명'].isnull() 을 활용해서 '상호명' 이라는 컬럼을 가져옵니다.


In [0]:
# 상호명과 브랜드명이 제대로 변경되었는지 확인합니다.
# 두 가지 컬럼을 가져옵니다.



In [0]:
# 브랜드별 수량을 세어봅니다.value_counts()를 활용해 주세요.


## scatterplot
* https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html#scatter-plot

In [0]:
# pandas의 scatterplot 을 그립니다.
# 스타벅스만 그리도록 되어 있는데 해당 코드를 활용해 이디야도 그리도록 합니다.
# ax = df_cafe[df_cafe["브랜드명"] == "스타벅스"].plot.scatter(
#     x="경도", y="위도", color="g", label="스타벅스")

In [0]:
# seaborn 을 통해 그려보세요.


In [0]:
geo_df = df_cafe

In [0]:
map = folium.Map(location=[geo_df['위도'].mean(), geo_df['경도'].mean()], 
                 zoom_start=12, tiles='Stamen Toner')

for n in geo_df.index:
    # 팝업에 들어갈 텍스트를 지정해 줍니다.
    popup_name = geo_df.loc[n, '상호명'] + ' - ' + geo_df.loc[n, '도로명주소']
    # 브랜드명에 따라 아이콘 색상을 달리해서 찍어줍니다.
    if geo_df.loc[n, '브랜드명'] == '스타벅스' :
        icon_color = 'green'
    else:
        icon_color = 'blue'    
        
    # folium.CircleMarker 혹은 folium.features.CircleMarker 오류가 날 경우
    # --> folium.vector_layer.CircleMarker 를 사용합니다.
    folium.CircleMarker(
        location=[geo_df.loc[n,'위도'], geo_df.loc[n, '경도']],
        radius=3,
        popup=popup_name,
        color= icon_color,
        fill=True,
        fill_color=icon_color
    ).add_to(map)


map

## 구별 브랜드별 점포수

In [0]:
df_cafe_brand = pd.DataFrame(
    df_cafe.groupby(['시군구명', '브랜드명'])['상호명'].count()
).reset_index()
df_cafe_brand.columns = ['구', '브랜드명', '매장수']
df_cafe_brand

## 막대그래프로 시각화 하기

In [0]:
# sns.barplot 을 활용해 df_cafe_brand 데이터 프레임으로 그려봅니다.
#  x='구', y='매장수', hue='브랜드명' 옵션을 사용합니다.



### Pandas의 groupby 활용하기

In [0]:
df_cafe.groupby(['시군구명'])['상호명'].count()

In [0]:
df_cafe_gu = pd.DataFrame(df_cafe.groupby(['시군구명'])['상호명'].count())
df_cafe_gu.head()

In [0]:
# df_cafe 데이터 프레임으로 시군구명, 브랜드명, 상호명 을 활용해 pivot_table 을 그립니다.
# 기본값이 평균이기 때문에 aggfunc="count" 를 지정합니다.
# 결과는 df_cafe_brand_gu에 저장합니다.

In [0]:
# Pandas 의 plot.bar를 그려봅니다.


## choropleth 그리기

* 구별로 매장수를 표현하기 위해 GeoJSON 파일 로드
    * 파일출처 : [southkorea/seoul-maps: Seoul administrative divisions in ESRI Shapefile, GeoJSON and TopoJSON formats.](https://github.com/southkorea/seoul-maps)

In [0]:
# 공유 가능한 링크로 파일 가져오기
url = 'https://drive.google.com/open?id=13j8-_XLdPe0pptsqu8-uyE-0Ym6V2jw5'
id = url.split('=')[1]
print(id)
downloaded = drive.CreateFile({'id':id}) 
# data 폴더에 파일을 관리하며, 폴더가 없다면 만들어서 파일을 관리하도록 한다.
downloaded.GetContentFile('seoul_municipalities_geo_simple.json')  

In [0]:
import json

geo_path = 'seoul_municipalities_geo_simple.json'
geo_json = json.load(open(geo_path, encoding='utf-8'))

### 스타벅스 이디야 카페 매장 합계 분포

In [0]:
map = folium.Map(location=[geo_df['위도'].mean(), geo_df['경도'].mean()], 
                 zoom_start=11)

map.choropleth(geo_data = geo_json,
               data = df_cafe_gu['상호명'],
               columns = [df_cafe_brand.index, df_cafe_gu['상호명']],
               fill_color = 'Purples',
               key_on = 'feature.properties.name',
               fill_opacity=0.7,
               line_opacity=0.2,
               highlight=True)


for n in geo_df.index:
    # 팝업에 들어갈 텍스트를 지정해 줍니다.
    popup_name = geo_df.loc[n, '상호명'] + ' - ' + geo_df.loc[n, '도로명주소']
    # 브랜드명에 따라 아이콘 색상을 달리해서 찍어줍니다.
    if geo_df.loc[n, '브랜드명'] == '스타벅스' :
        icon_color = 'green'
    else:
        icon_color = 'blue'    
    
    # folium.CircleMarker 혹은 folium.features.CircleMarker 오류가 날 경우
    # --> folium.vector_layer.CircleMarker 를 사용합니다. 
    folium.CircleMarker(
        location=[geo_df.loc[n,'위도'], geo_df.loc[n, '경도']],
        radius=3,
        popup=popup_name,
        color= icon_color,
        fill=True,
        fill_color=icon_color
    ).add_to(map)

map

### 스타벅스 매장 분포

In [0]:
df_cafe_temp = df_cafe[df_cafe['브랜드명'] == '스타벅스']
df_cafe_starbucks = pd.DataFrame(df_cafe_temp.groupby(['시군구명'])['상호명'].count())
df_cafe_starbucks.columns = ['매장수']
df_cafe_starbucks.head()

In [0]:
map = folium.Map(location=[geo_df['위도'].mean(), geo_df['경도'].mean()], 
                 zoom_start=11, tiles='Stamen Toner')

map.choropleth(geo_data = geo_json,
               data = df_cafe_starbucks['매장수'],
               columns = [df_cafe_starbucks.index, df_cafe_starbucks['매장수']],
               fill_color = 'YlGn',
               key_on = 'feature.properties.name',
               fill_opacity=0.7,
               line_opacity=0.2,
               highlight=True)


for n in geo_df.index:
    # 팝업에 들어갈 텍스트를 지정해 줍니다.
    popup_name = geo_df.loc[n, '상호명'] + ' - ' + geo_df.loc[n, '도로명주소']
    # 브랜드명에 따라 아이콘 색상을 달리해서 찍어줍니다.
    if geo_df.loc[n, '브랜드명'] == '스타벅스' :
        icon_color = 'green'    

            
        # folium.CircleMarker 혹은 folium.features.CircleMarker 오류가 날 경우
        # --> folium.vector_layer.CircleMarker 를 사용합니다.
        folium.CircleMarker(
            location=[geo_df.loc[n,'위도'], geo_df.loc[n, '경도']],
            radius=3,
            popup=popup_name,
            color= icon_color,
            fill=True,
            fill_color=icon_color
        ).add_to(map)


map

### 이디야 매장 분포

In [0]:
df_cafe_temp = df_cafe[df_cafe['브랜드명'] == '이디야']
df_cafe_ediya = pd.DataFrame(df_cafe_temp.groupby(['시군구명'])['상호명'].count())
df_cafe_ediya.columns = ['매장수']
df_cafe_ediya.head()

In [0]:
map = folium.Map(location=[geo_df['위도'].mean(), geo_df['경도'].mean()], 
                 zoom_start=11, tiles='Stamen Toner')

map.choropleth(geo_data = geo_json,
               data = df_cafe_ediya['매장수'],
               columns = [df_cafe_ediya.index, df_cafe_ediya['매장수']],
               fill_color = 'PuBu',
               key_on = 'feature.properties.name',
               fill_opacity=0.7,
               line_opacity=0.2,
               highlight=True)


for n in geo_df.index:
    # 팝업에 들어갈 텍스트를 지정해 줍니다.
    popup_name = geo_df.loc[n, '상호명'] + ' - ' + geo_df.loc[n, '도로명주소']
    # 브랜드명에 따라 아이콘 색상을 달리해서 찍어줍니다.
    if geo_df.loc[n, '브랜드명'] == '이디야' :
        icon_color = 'blue'    

            
        # folium.CircleMarker 혹은 folium.features.CircleMarker 오류가 날 경우
        # --> folium.vector_layer.CircleMarker 를 사용합니다.
        folium.CircleMarker(
            location=[geo_df.loc[n,'위도'], geo_df.loc[n, '경도']],
            radius=3,
            popup=popup_name,
            color= icon_color,
            fill=True,
            fill_color=icon_color
        ).add_to(map)


map

## 매장수 크기를 반영해 CircleMaker 그리기
### Pandas 의 reshaping data 활용하기

<img src="https://i.imgur.com/IIhU0nF.png">

* 출처 : https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf
* reshaping 관련 문서 : https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html

In [0]:
df_cafe_brand.head()

In [0]:
# 스타벅스 이디야가 컬럼으로 오도록 만듭니다.
df_cafe_brand_vs = df_cafe_brand.pivot_table(index='구',
                                             columns='브랜드명',
                                             values='매장수')
df_cafe_brand_vs.columns = ['스타벅스', '이디야']
df_cafe_brand_vs.head()

In [0]:
# 함수를 만듭니다. 
# 스타벅스와 이디야의 매장수를 비교해서 스타벅스가 더 많다면 1을 반환 아니면 0을 반환하도록 합니다.
# def store_count(store):

In [0]:
df_cafe_brand_vs['매장수비교'] = df_cafe_brand_vs.apply(store_count, axis=1)
df_cafe_brand_vs.head()

In [0]:
# 위와 같이 표현할 수도 있지만 아래와 같이 lambda로 간결하게 표현할 수도 있습니다.
df_cafe_brand_vs['매장수비교'] = df_cafe_brand_vs.apply( \
    lambda x : 1 if x['스타벅스'] > x['이디야'] else 0, axis=1)
df_cafe_brand_vs.head()

## 지도표현을 위해 구별 위경도 평균 구하기

In [0]:
lng_list = []
lat_list = []
for gu in df_cafe_brand_vs.index:
#     lat = 구별 평균을 구합니다.
#     lng = 구별 평균을 구합니다.
    lat_list.append(lat)
    lng_list.append(lng)
    
# df_cafe_brand_vs['위도'] = 리스트에 저장된 값을 넣어줍니다. 
# df_cafe_brand_vs['경도'] = 리스트에 저장된 값을 넣어줍니다.
df_cafe_brand_vs.head()

In [0]:
# CircleMarker의 radius 지정시 다음과 같은 타입오류가 나서 float type 으로 변경
# TypeError: Object of type 'int64' is not JSON serializable

df_cafe_brand_vs['스타벅스'] = df_cafe_brand_vs['스타벅스'].astype(float)
df_cafe_brand_vs['이디야'] = df_cafe_brand_vs['이디야'].astype(float)
df_cafe_brand_vs.info()

In [0]:
map = folium.Map(location=[geo_df['위도'].mean(), geo_df['경도'].mean()], 
                 zoom_start=11, tiles='Stamen Toner')

map.choropleth(geo_data = geo_json,
               data = df_cafe_brand_vs['매장수비교'],
               columns = [df_cafe_brand_vs.index, 
                          df_cafe_brand_vs['매장수비교']],
               fill_color = 'BuGn',
               key_on = 'feature.properties.name',
               fill_opacity=0.7,
               line_opacity=0.2,
               highlight=True)


for gu in df_cafe_brand_vs.index:
        
    for cafe in ['스타벅스', '이디야']:
        
        cafe_count = df_cafe_brand_vs.loc[gu, cafe]
        msg = f'{gu} {cafe} 매장수 : {cafe_count:.0f}'
        
        icon_color = 'blue'
        if cafe == '스타벅스': 
            icon_color = 'green'
        
        
        # folium.CircleMarker 혹은 folium.features.CircleMarker 오류가 날 경우
        # --> folium.vector_layer.CircleMarker 를 사용합니다.
        folium.CircleMarker(
            location = [df_cafe_brand_vs.loc[gu,'위도'], 
                      df_cafe_brand_vs.loc[gu, '경도']],
            radius = cafe_count,
            color = icon_color,
            popup = msg,
            fill = True,
            fill_color = icon_color
        ).add_to(map)

map

#### 지도 출력 시 색상 참고
* http://colorbrewer2.org/

## folium 공부를 위해 더 읽어보면 좋을 것 
* https://nbviewer.jupyter.org/github/python-visualization/folium/tree/master/examples/
* https://python-visualization.github.io/folium/quickstart.html#Choropleth-maps
* [PinkWink의 데이터 사이언스 관련 책 - 파이썬으로 데이터 주무르기](https://pinkwink.kr/1070)

## 카토그램 
* [총선⑫ 인구 비례로 본 당선 지도…‘카토그램’으로 살펴본 당선 현황 > 데이터룸 > 정치 > 뉴스 | KBSNEWS](http://news.kbs.co.kr/news/view.do?ncd=3264019)
* [The Housing Value of Every County in the U.S. - Metrocosm](http://metrocosm.com/the-housing-value-of-every-county-in-the-u-s/)